<a href="https://colab.research.google.com/github/NikitaSUAI/EmotionRecognition/blob/main/Load_OMGEmmotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load dataset and libs

In [ ]:
! git clone https://github.com/knowledgetechnologyuhh/OMGEmotionChallenge.git

Cloning into 'OMGEmotionChallenge'...
remote: Enumerating objects: 20860, done.
remote: Total 20860 (delta 0), reused 0 (delta 0), pack-reused 20860
Receiving objects: 100% (20860/20860), 2.05 MiB | 11.51 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Checking out files: 100% (294589/294589), done.


In [ ]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 1.4 MB/s 


# Find all paths to the dataset

In [ ]:
import pandas as pd
from pathlib import Path
import shutil
from tqdm.notebook import tqdm

LOSTED_VIDEO = ["c0f84b343_1", "5ba7890ab_1"]

TRAIN_VIDEO = Path("/content/OMGEmotionChallenge/omg_TrainVideos.csv")
TRAIN_TEXT = Path("/content/OMGEmotionChallenge/omg_TrainTranscripts.csv")

TEST_VIDEO = Path("/content/OMGEmotionChallenge/omg_TestVideos_WithLabels.csv")
TEST_TEXT = Path("/content/OMGEmotionChallenge/omg_TestTranscripts.tsv")

VAL_VIDEO = Path("/content/OMGEmotionChallenge/omg_ValidationVideos.csv")
VAL_TEXT = Path("/content/OMGEmotionChallenge/omg_ValidationTranscripts.csv")

BASE_PATH = Path("/content/data")
BASE_PATH.mkdir(exist_ok=True)

train_video = pd.read_csv(TRAIN_VIDEO)
train_text = pd.read_csv(TRAIN_TEXT)

test_video = pd.read_csv(TEST_VIDEO)
test_video = test_video[test_video.video != "5ba7890ab_1"]
test_text = pd.read_csv(TEST_TEXT)
test_text = test_text[test_text.video != "5ba7890ab_1"]


val_video = pd.read_csv(VAL_VIDEO)
val_text = pd.read_csv(VAL_TEXT)

dataset = {
    "train": (train_video, train_text), 
    "val": (val_video, val_text),
    "test": (test_video, test_text)
}

# Upload audio files and bring the files into a convenient format

## Code for download and test for it

In [ ]:
from pytube import YouTube, helpers
from pytube.exceptions import VideoPrivate, VideoUnavailable
import os

import sys
from moviepy.editor import VideoFileClip

def downloadAndExtractAudio(videourl, path, video_id, utt):
    out_utt = []
    tmp_path = Path("/content/tmp/")
    yt = YouTube(videourl)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    yt.download(tmp_path)
    for video_path in tmp_path.glob("*.mp4"):
      path = path / f"{video_id}"
      path.mkdir(exist_ok=True)
      video = VideoFileClip(str(video_path))
      for seq in utt:
        name, start, stop = seq
        audio = video.audio.subclip(start,stop)
        local_path = path / f"{name.strip('.mp4')}.wav"
        audio.write_audiofile(local_path, verbose=False, progress_bar=False)

        out_utt.append({"utterance": name, "path":str(Path(*local_path.parts[3:]))})
      video_path.unlink()
      return pd.DataFrame(out_utt).set_index(["utterance"])

# ------------------------------------------------------------------------------      
# need only for test
out_train =  Path("/content/data/train")
out_train.mkdir(exist_ok=True)
link = 	"https://www.youtube.com/watch?v=_bg0TrqHcBs"
video_id = "1f61459b0"
utt = (
    ("utterance_1.mp4", 0.000000, 16.231716),
    ("utterance_2.mp4", 16.565016, 21.864488),
)
downloadAndExtractAudio(link, out_train, video_id, utt)
# ------------------------------------------------------------------------------

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2064384/45929032 bytes (4.5%)4947968/45929032 bytes (10.8%)7610368/45929032 bytes (16.6%)10485760/45929032 bytes (22.8%)13590528/45929032 bytes (29.6%)16506880/45929032 bytes (35.9%)19677184/45929032 bytes (42.8%)22740992/45929032 bytes (49.5%)25690112/45929032 bytes (55.9%)28778496/45929032 bytes (62.7%)31817728/45929032 bytes (69.3%)34955264/45929032 bytes (76.1%)

,path
utterance,
utterance_1.mp4,train/1f61459b0/utterance_1.wav
utterance_2.mp4,train/1f61459b0/utterance_2.wav


## Save to

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

TRAIN_PATH = BASE_PATH.parent / "train_set.csv"
VAL_PATH = BASE_PATH.parent / "val_set.csv"
TEST_PATH = BASE_PATH.parent / "/test_set.csv"

!cp -r '/content/drive/MyDrive/OMG_EMO' 'OMG_EMO'
peths = {
    "train" : TRAIN_PATH,
    "val" : VAL_PATH,
    "test" : TEST_PATH
}
res = {}

try:
  for key, val in peths.items():
    res[key] = [pd.read_csv(val), ]
except :
  ...

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open("/content/drive/MyDrive/OMG_EMO/error_list.jsom", "r") as f:
  error_list = json.load( f)

In [ ]:
tmp = {}
for key, val in error_list.items():
  tmp[key] = {}
  for string in val:
    link, msg = string.split(" : ")
    tmp[key][link] = msg
error_list = tmp

In [ ]:
from urllib.error import HTTPError
import json


proxy_handlers = [
                  {"https": "aS845M:onfoft@81.177.180.147:11429"},
                  {"https": "aS845M:onfoft@81.177.180.147:11428"},
                  {"https": "aS845M:onfoft@81.177.180.147:11427"},
                  {"https": "aS845M:onfoft@81.177.180.147:11426"},
                  {"https": "aS845M:onfoft@81.177.180.147:11433"},
                  {"https": "aS845M:onfoft@81.177.180.147:11432"},
                  {"https": "aS845M:onfoft@81.177.180.147:11431"},
                  {"https": "aS845M:onfoft@81.177.180.147:11430"},
                  {"https": "aS845M:onfoft@81.177.180.147:11425"},
]
it = 0

for data, video, text in dataset:
  out_folder = BASE_PATH / data
  out_folder.mkdir(exist_ok=True)
  unique_id = video.video.unique()
  times = 0
  for video_id in tqdm(unique_id):
    try:
      if video_id in res[data][0].index:
        continue
    except KeyError:
      ...
    cur_video = video[video.video == video_id]
    cur_text = text[text.video == video_id]
    tmp = cur_video.set_index(["video", "utterance"]).join(cur_text[["video", "utterance", "transcript"]].set_index(["video", "utterance"]))

    if cur_video.shape[0] != cur_text.shape[0]:
      raise Exception("Something didn't go according to plan")

    link = cur_video.link.unique()
    if link.shape[0] > 1:
      raise Exception("Something didn't go according to plan")
    else:
      link = link[0]
    utt = [(row.utterance, row.start, row.end) for idx, row in cur_video.iterrows()]
    
    try:
      utt = downloadAndExtractAudio(link, out_folder, video_id, utt)
      tmp = tmp.join(utt)
      shutil.copytree(out_folder, f"/content/drive/MyDrive/OMG_EMO/data/{data}")
    except HTTPError:
      helpers.install_proxy(proxy_handlers[it % len(proxy_handlers)])
      it += 1
      utt = downloadAndExtractAudio(link, out_folder, video_id, utt)
      tmp = tmp.join(utt)
      shutil.copytree(out_folder/video_id, f"/content/drive/MyDrive/OMG_EMO/data/{data}/{video_id}")
    except Exception as e:
      error_list[data][link] = str(e)
      with open("/content/drive/MyDrive/OMG_EMO/error_list.jsom", "w") as f:
        json.dump(error_list, f)
    
    try:
      res[data].append(tmp)
      res[data] = pd.concat(res[data])
      res[data] = [res[data], ]
    except KeyError:
      res[data] = [tmp, ]
      res[data] = pd.concat(res[data])
      res[data] = [res[data], ]
    res[data][0].to_csv(peths[data])
    shutil.copyfile(peths[data], f"/content/drive/MyDrive/OMG_EMO/{peths[data].name}")

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/203 [00:00<?, ?it/s]

In [ ]:
shutil.copytree(out_folder/video_id, f"/content/drive/MyDrive/OMG_EMO/data/{data}/{video_id}")

'/content/drive/MyDrive/OMG_EMO/data/train/664ea0ccc_1'

In [ ]:
! cp -r /content/data/* /content/drive/MyDrive/OMG_EMO/data

In [ ]:
! rm -rf /content/drive/MyDrive/OMG_EMO/data/data

In [ ]:
new_error_list = {
    "train" : [],
    "val" : [],
    "test" : []
}

for dset, errors in error_list.items():
  video_ds, text_ds = dataset[dset]
  out_folder = BASE_PATH / dset
  out_folder.mkdir(exist_ok=True)
  for link, err_text in tqdm(errors.items()):
    cur_video = video_ds[video_ds.link == link]
    video_id = cur_video.video.unique()[0]
    cur_text = text[text.video == video_id]

    print()
    tmp = cur_video.set_index(["video", "utterance"]).join(cur_text[["video", "utterance", "transcript"]].set_index(["video", "utterance"]))

    if cur_video.shape[0] != cur_text.shape[0]:
      raise Exception("Something didn't go according to plan")

    link = cur_video.link.unique()[0]

    utt = [(row.utterance, row.start, row.end) for idx, row in cur_video.iterrows()]
    
    try:
      utt = downloadAndExtractAudio(link, out_folder, video_id, utt)
      tmp = tmp.join(utt)
      shutil.copytree(out_folder/video_id, f"/content/drive/MyDrive/OMG_EMO/data/{data}/{video_id}")
    except HTTPError:
      helpers.install_proxy(proxy_handlers[it % len(proxy_handlers)])
      it += 1
      utt = downloadAndExtractAudio(link, out_folder, video_id, utt)
      tmp = tmp.join(utt)
      shutil.copytree(out_folder/video_id, f"/content/drive/MyDrive/OMG_EMO/data/{data}/{video_id}")
    except Exception as e:
      new_error_list[data][link] = str(e)
      with open("/content/drive/MyDrive/OMG_EMO/error_list_n1.jsom", "w") as f:
        json.dump(new_error_list, f)
    
    try:
      res[data].append(tmp)
      res[data] = pd.concat(res[data])
      res[data] = [res[data], ]
    except KeyError:
      res[data] = [tmp, ]
      res[data] = pd.concat(res[data])
      res[data] = [res[data], ]
    res[data][0].to_csv(peths[data])
    shutil.copyfile(peths[data], f"/content/drive/MyDrive/OMG_EMO/{peths[data].name}")

  0%|          | 0/125 [00:00<?, ?it/s]

Exception: ignored